# Understanding Data Check Actions

EvalML streamlines the creation and implementation of machine learning models for tabular data. One of the many features it offers is [data checks](https://evalml.alteryx.com/en/stable/user_guide/data_checks.html), which help determine the health of our data before we train a model on it. These data checks have associated actions with them and will be shown in this notebook. In our default data checks, we have the following checks:

- `NullDataCheck`: Checks whether the rows or columns are highly null

- `IDColumnsDataCheck`: Checks for columns that could be ID columns

- `TargetLeakageDataCheck`: Checks if any of the input features have high association with the targets

- `InvalidTargetDataCheck`: Checks if there are null or other invalid values in the target

- `NoVarianceDataCheck`: Checks if either the target or any features have no variance

- `NaturalLanguageNaNDataCheck`: Checks if any natural language columns have missing data

- `DateTimeNaNDataCheck`: Checks if any datetime columns have missing data


EvalML has additional data checks that can be seen [here](https://evalml.alteryx.com/en/stable/api_index.html#data-checks), with usage examples [here](https://evalml.alteryx.com/en/stable/user_guide/data_checks.html). Below, we will walk through usage of EvalML's default data checks and actions.


First, we import the necessary requirements to demonstrate these checks.

In [ ]:
import woodwork as ww
import pandas as pd
from evalml import AutoMLSearch
from evalml.demos import load_fraud
from evalml.preprocessing import split_data

Let's look at the input feature data. EvalML uses the [Woodwork](https://woodwork.alteryx.com/en/stable/) library to represent this data. The demo data that EvalML returns is a Woodwork DataTable and DataColumn.

In [ ]:
X, y = load_fraud(n_rows=1500)
X.head()

## Adding noise and unclean data

This data is already clean and compatible with EvalML's ``AutoMLSearch``. In order to demonstrate EvalML default data checks, we will add the following:

- A column of mostly null values (<0.5% non-null)

- A column with low/no variance

- A row of null values

- A missing target value


We will add the first two columns to the whole dataset and we will only add the last two to the training data. Note: these only represent some of the scenarios that EvalML default data checks can catch.

In [ ]:
# add a column with no variance in the data
X['no_variance'] = [1 for _ in range(X.shape[0])]

# add a column with >99.5% null values
X['mostly_nulls'] = [None] * (X.shape[0] - 5) + [i for i in range(5)]

# since we changed the data, let's reinitialize the woodwork datatable
X.ww.init()
# let's split some training and validation data
X_train, X_valid, y_train, y_valid = split_data(X, y, problem_type='binary')

In [ ]:
# let's copy the datetime at row 1 for future use
date = X_train.iloc[1]['datetime']

# make row 1 all nan values
X_train.iloc[1] = [None] * X_train.shape[1]

# make one of the target values null
y_train[990] = None

X_train.ww.init()
y_train = ww.init_series(y_train)
# Let's take another look at the new X_train data
X_train

If we call `AutoMLSearch.search()` on this data, the search will fail due to the columns and issues we've added above. Note: we use a try/except here to catch the resulting ValueError that AutoMLSearch raises.

In [ ]:
automl = AutoMLSearch(X_train=X_train, y_train=y_train, problem_type='binary')
try:
    automl.search()
except ValueError as e:
    # to make the error message more distinct
    print("=" * 80, "\n")
    print("Search errored out! Message received is: {}".format(e))
    print("=" * 80, "\n")

We can use the `search_iterative()` function provided in EvalML to determine what potential health issues our data has. We can see that this [search_iterative](https://evalml.alteryx.com/en/latest/autoapi/evalml/automl/index.html#evalml.automl.search_iterative) function is a public method available through `evalml.automl` and is different from the [search](https://evalml.alteryx.com/en/stable/autoapi/evalml/automl/index.html#evalml.automl.AutoMLSearch) function of the `AutoMLSearch` class in EvalML. This `search_iterative()` function allows us to run the default data checks on the data, and, if there are no errors, automatically runs `AutoMLSearch.search()`.

In [ ]:
from evalml.automl import search_iterative
automl, messages = search_iterative(X_train, y_train, problem_type='binary')
automl, messages

The return value of the `search_iterative` function above is a tuple. The first element is the `AutoMLSearch` object if it runs (and `None` otherwise), and the second element is a dictionary of potential warnings and errors that the default data checks find on the passed-in `X` and `y` data. In this dictionary, warnings are suggestions that the data checks give that can useful to address to make the search better but will not break `AutoMLSearch`. On the flip side, errors indicate issues that will break `AutoMLSearch` and need to be addressed by the user.

Above, we can see that there were errors so search did not automatically run.

## Addressing warnings and errors

We can automatically address the warnings and errors returned by ``search_iterative`` by using ``make_pipeline_from_actions``, a utility method that creates a pipeline that will automatically clean up our data. We just need to pass this method ``DataCheckAction`` objects and our problem type.

In [ ]:
# Each data check message may return a list of the dictionary version of each action that can be taken to clean up the data
action_options = []
for message in messages:
    action_options.extend([option for option in message["action_options"]])

action_options

In [ ]:
from evalml.pipelines.utils import make_pipeline_from_actions, get_actions_from_option_defaults
from evalml.data_checks import DataCheckActionOption

# Convert dictionary form of actions returned from data check output dictionary as DataCheckAction objects
actions = [
        DataCheckActionOption.convert_dict_to_option(action)
        for action in action_options
]

actions = get_actions_from_option_defaults(actions)

actions_pipeline = make_pipeline_from_actions("binary", actions)
actions_pipeline.fit(X_train, y_train)
X_train_cleaned, y_train_cleaned = actions_pipeline.transform(X_train, y_train)
print("The new length of X_train is {} and y_train is {}".format(len(X_train_cleaned),len(X_train_cleaned)))

Now, we can run ``search_iterative`` to completion.

In [ ]:
results_cleaned = search_iterative(X_train_cleaned, y_train_cleaned, problem_type='binary')

Note that this time, we get an `AutoMLSearch` object returned to us as the first element of the tuple. We can use and inspect the `AutoMLSearch` object as needed.

In [ ]:
automl_object = results_cleaned[0]
automl_object.rankings

If we check the second element in the tuple, we can see that there are no longer any warnings or errors detected!

In [ ]:
data_check_results = results_cleaned[1]
data_check_results

## Only addressing DataCheck errors


Previously, we used `make_pipeline_from_actions` to address all of the warnings and errors returned by `search_iterative`. We will now show how we can also manually address errors to allow AutoMLSearch to run, and how ignoring warnings will come at the expense of performance.

We can print out the errors first to make it easier to read, and then we'll create new features and targets from the original training data.

In [ ]:
errors = [message for message in messages if message['level'] == 'error']
errors

In [ ]:
# copy the DataTables to new variables
X_train_no_errors = X_train.copy()
y_train_no_errors = y_train.copy()

# We address the errors by looking at the resulting dictionary errors listed

# first, let's address the `TARGET_HAS_NULL` error
y_train_no_errors.fillna(False, inplace=True)

# here, we address the `NO_VARIANCE` error 
X_train_no_errors.drop("no_variance", axis=1, inplace=True)

# lastly, we address the `DATETIME_HAS_NAN` error with the date we had saved earlier
X_train_no_errors.iloc[1, 2] = date

# let's reinitialize the Woodwork DataTable
X_train_no_errors.ww.init()
X_train_no_errors.head()

We can now run search on `X_train_no_errors` and `y_train_no_errors`. Note that the search here doesn't fail since we addressed the errors, but there will still exist warnings in the returned tuple. This search allows the `mostly_nulls` column to remain in the features during search.

In [ ]:
results_no_errors = search_iterative(X_train_no_errors, y_train_no_errors, problem_type='binary')
results_no_errors

## Comparing removing only errors versus removing both warnings and errors
Let's see the differences in model performance when we remove only errors versus remove both warnings and errors. To do this, we compare the performance of the best pipelines on the validation data. Remember that in the search where we only address errors, we still have the `mostly_nulls` column present in the data, so we leave that column in the validation data for its respective search. We drop the other `no_variance` column from both searches.

Additionally, we do some logical type setting since we had added additional noise to just the training data. This allows the data to be of the same types in both training and validation.

In [ ]:
# drop the no_variance column
X_valid.drop("no_variance", axis=1, inplace=True)

# logical type management
X_valid.ww.init(logical_types={"customer_present": "Categorical"})
y_valid = ww.init_series(y_valid, logical_type="Categorical")

best_pipeline_no_errors = results_no_errors[0].best_pipeline
print("Only dropping errors:", best_pipeline_no_errors.score(X_valid, y_valid, ["Log Loss Binary"]), "\n")

# drop the mostly_nulls column and reinitialize the DataTable
X_valid.drop("mostly_nulls", axis=1, inplace=True)
X_valid.ww.init()

best_pipeline_clean = results_cleaned[0].best_pipeline
print("Addressing all actions:", best_pipeline_clean.score(X_valid, y_valid, ["Log Loss Binary"]), "\n")

We can compare the differences in model performance when we address all action items (warnings and errors) in comparison to when we only address errors. While it isn't guaranteed that addressing all actions will always have better performance, we do recommend doing so since we only raise these issues when we believe the features have problems that could negatively impact or not benefit the search.